In [6]:
silver_df = spark.sql("SELECT * FROM LH_Health.Silver.silver_health_data ")
display(silver_df)

StatementMeta(, f313a989-3808-48d6-87f0-fef42f09f50d, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ab143364-56fb-4ab8-a850-663e0831e64b)

In [7]:
dim_hospital = silver_df.select('Facility_Name').distinct().withColumnRenamed('Facility_Name','Hospital_Name')

dim_hospital.write.format('delta').saveAsTable('LH_Health.Gold.dim_hospital')

StatementMeta(, f313a989-3808-48d6-87f0-fef42f09f50d, 9, Finished, Available, Finished)

In [12]:
from pyspark.sql.functions import avg , round


gold_summary = silver_df.groupBy("Facility_Name","Age_Group")\
    .agg(round(avg("Total_Charges"),2).alias("Avg_Cost_Per_Stay"),
    round(avg("Length_of_stay"),2).alias("Avg_Length_Stay"))

display(gold_summary)

StatementMeta(, f313a989-3808-48d6-87f0-fef42f09f50d, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ec076e29-1478-4800-9a3e-460b32e55d9b)

In [13]:
gold_summary.write.format('delta').saveAsTable('LH_Health.Gold.gold_hospital_summary')

StatementMeta(, f313a989-3808-48d6-87f0-fef42f09f50d, 15, Finished, Available, Finished)

In [16]:
from pyspark.sql.functions import when , col

fact_enriched = silver_df\
    .withColumn("Profit_Margin",col("Total_Charges")-col("Total_Costs"))\
    .withColumn("High_Cost_Flag",when(col("Total_Charges")>50000,"Y").otherwise("N"))

display(fact_enriched)


StatementMeta(, f313a989-3808-48d6-87f0-fef42f09f50d, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0204fe57-a85d-4e14-be2a-09603ecf4747)

In [17]:
fact_enriched.write.format('delta').saveAsTable('LH_Health.Gold.Fact_Discharge_Enriched')

StatementMeta(, f313a989-3808-48d6-87f0-fef42f09f50d, 19, Finished, Available, Finished)